If you want to be in the `PedestrianAvoidancePOMDP` environment, run the following cell. If the package has been added to the main julia environment, it is not necessary.

In [1]:
] activate ../ 

In [2]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using PedestrianAvoidancePOMDP
using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/markuss/.julia/compiled/v1.0/PedestrianAvoidancePOMDP/UuJ2S.ji for PedestrianAvoidancePOMDP [a80ff0fe-e6a0-11e8-3b05-15413bd5ea7a]
└ @ Base loading.jl:1190
┌ Warning: Package PedestrianAvoidancePOMDP does not have EmergencyBrakingSystem in its dependencies:
│ - If you have PedestrianAvoidancePOMDP checked out for development and have
│   added EmergencyBrakingSystem as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PedestrianAvoidancePOMDP
└ Loading EmergencyBrakingSystem into PedestrianAvoidancePOMDP from project dependency, future warnings for PedestrianAvoidancePOMDP are suppressed.


In [3]:
policy = load("../policy/policy.jld2")["policy"]; # need to be updated

In [4]:
pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)

parameters = CrosswalkParams()
parameters.roadway_length = 400.0
parameters.obstacles_visible = false

obstacle_offset = -2
obstacle_1 = ConvexPolygon([VecE2(34, obstacle_offset), VecE2(34, obstacle_offset-3), VecE2(46.5, obstacle_offset-3), VecE2(46.5, obstacle_offset)],4)
obstacle_2 = ConvexPolygon([VecE2(34, +4.5), VecE2(34, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
parameters.obstacles = [obstacle_1, obstacle_2]
#parameters.obstacles = [obstacle_2]

env = CrosswalkEnv(parameters)
pomdp.env = env


ego_id = 1
ped_id = 2
ped2_id = 3


ego_v = 11.11
ped_v = 5.4/3.6
hitpoint = 300             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -3.5
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;


if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -2.0
    ped_v = 0.0
end

ped_y_start = -4.
ped_v = 1.0
ego_y = 0.0


# Car definition
ego_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
ego = Vehicle(ego_initial_state, VehicleDef(), 1)

# Pedestrian definition 
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

ped2_v = 0.0
ped2 = Vehicle(VehicleState(VecSE2(54., 5., 0.78), env.crosswalk, env.roadway, ped2_v), AutomotivePOMDPs.PEDESTRIAN_DEF, ped2_id)


scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



cam = FitToContentCamera(0.);
timestep = 0.2


pos_noise = 0.001
vel_noise = 0.000
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 


# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

pomdp.desired_velocity = ego_v
pomdp.ΔT = timestep
models[ego_id] = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),
    env=env,
    sensor=sensor,
    obstacles=env.obstacles,
    timestep=timestep,
    pomdp=pomdp,
    policy=policy,
    updater=updater,
    ego_vehicle=ego,
    desired_velocity=ego_v,
    b=PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, ego_y, ego_v)
)

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.0001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.0001) # dumb model



nticks = 110
rec = SceneRecord(nticks+1, timestep)

risk = Float64[]
sensor_observations = [Vehicle[]]
belief = Dict{Int64, SingleOCFBelief}[]
ego_vehicle = Vehicle[]
action_pomdp = SingleOCFAction[]

obs_callback = (ObservationCallback(risk,sensor_observations, belief, ego_vehicle, action_pomdp),)

@elapsed begin
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
end




--------------------------POMDP high level planner----------------------- t: 0.0
EGO: x/y:-11.25333333333333 / 0.0 v: 11.11
Vehicle(2, VehicleState(VecSE2({50.000, -4.000}, 1.571), Frenet(RoadIndex({1, 0.300000}, {2, 1}), 6.000, 150.000, 0.000), 1.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.000}, 0.780), Frenet(RoadIndex({1, 0.750000}, {2, 1}), 15.000, 146.000, -0.791), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
b-length: 20
SingleOCFState[[0.0, 12.0, 44.0, -5.0, 1.57, 0.0], [0.0, 12.0, 46.0, -5.0, 1.57, 0.0], [0.0, 12.0, 48.0, -5.0, 1.57, 0.0], [0.0, 12.0, 50.0, -5.0, 1.57, 0.0], [0.0, 12.0, 44.0, -5.0, 1.57, 0.5], [0.0, 12.0, 46.0, -5.0, 1.57, 0.5], [0.0, 12.0, 48.0, -5.0, 1.57, 0.5], [0.0, 12.0, 50.0, -5.0, 1.57, 0.5], [0.0, 12.0, 44.0, -5.0, 1.57, 1.0], [0.0, 12.0, 46.0, -5.0, 1.57, 1.0], [0.0, 12.0, 48.0, -5.0, 1.57, 1.0], [0.0, 12.0, 50.0, -5.0, 1.57, 1.0], [0.0, 12.0, 44.0, -5.0, 1.57, 1.5], [0.0, 12.0, 46.0, -5.0, 1.57, 1.5], [0.0, 1

SingleOCFState[[0.0, 9.0, 44.0, -5.0, 1.57, 0.0], [0.0, 9.0, 44.0, -4.0, 1.57, 0.0], [0.0, 9.0, 44.0, -5.0, 1.57, 0.5], [0.0, 9.0, 44.0, -4.0, 1.57, 0.5], [0.0, 9.0, 44.0, -3.0, 1.57, 0.5], [0.0, 9.0, 44.0, -5.0, 1.57, 1.0], [0.0, 9.0, 44.0, -4.0, 1.57, 1.0], [0.0, 9.0, 44.0, -3.0, 1.57, 1.0], [0.0, 9.0, 44.0, -2.0, 1.57, 1.0], [0.0, 9.0, 44.0, -5.0, 1.57, 1.5], [0.0, 9.0, 44.0, -4.0, 1.57, 1.5], [0.0, 9.0, 44.0, -3.0, 1.57, 1.5], [0.0, 9.0, 44.0, -2.0, 1.57, 1.5], [0.0, 9.0, 44.0, -5.0, 1.57, 2.0], [0.0, 9.0, 44.0, -4.0, 1.57, 2.0], [0.0, 9.0, 44.0, -3.0, 1.57, 2.0], [0.0, 9.0, 44.0, -2.0, 1.57, 2.0]]
---------> modified: [0.0, 9.0, 50.0, 5.0, 0.0, 0.0]
dict-modified: Dict(2=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 9.0, 44.0, -5.0, 1.57, 0.0], [0.0, 9.0, 44.0, -4.0, 1.57, 0.0], [0.0, 9.0, 44.0, -5.0, 1.57, 0.5], [0.0, 9.0, 44.0, -4.0, 1.57, 0.5], [0.0, 9.0, 44.0, -3.0, 1.57, 0.5], [0.0, 9.0, 44.0, -5.0, 1.57, 1.0], [0.0, 9.0, 44.0, -4.0, 1.57, 1.0], [0

dict-modified: Dict(2=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 7.0, 46.0, -4.0, 1.57, 0.0], [0.0, 7.0, 48.0, -4.0, 1.57, 0.0], [0.0, 7.0, 50.0, -4.0, 1.57, 0.0], [0.0, 7.0, 46.0, -3.0, 1.57, 0.0], [0.0, 7.0, 48.0, -3.0, 1.57, 0.0], [0.0, 7.0, 50.0, -3.0, 1.57, 0.0], [0.0, 7.0, 46.0, -2.0, 1.57, 0.0], [0.0, 7.0, 48.0, -2.0, 1.57, 0.0], [0.0, 7.0, 50.0, -2.0, 1.57, 0.0], [0.0, 7.0, 46.0, -4.0, 1.57, 0.5], [0.0, 7.0, 48.0, -4.0, 1.57, 0.5], [0.0, 7.0, 50.0, -4.0, 1.57, 0.5], [0.0, 7.0, 46.0, -3.0, 1.57, 0.5], [0.0, 7.0, 48.0, -3.0, 1.57, 0.5], [0.0, 7.0, 50.0, -3.0, 1.57, 0.5], [0.0, 7.0, 46.0, -2.0, 1.57, 0.5], [0.0, 7.0, 48.0, -2.0, 1.57, 0.5], [0.0, 7.0, 50.0, -2.0, 1.57, 0.5], [0.0, 7.0, 46.0, -4.0, 1.57, 1.0], [0.0, 7.0, 48.0, -4.0, 1.57, 1.0], [0.0, 7.0, 50.0, -4.0, 1.57, 1.0], [0.0, 7.0, 46.0, -3.0, 1.57, 1.0], [0.0, 7.0, 48.0, -3.0, 1.57, 1.0], [0.0, 7.0, 50.0, -3.0, 1.57, 1.0], [0.0, 7.0, 46.0, -2.0, 1.57, 1.0], [0.0, 7.0, 48.0, -2.0, 1.57, 1.0], 

b-length: 89
SingleOCFState[[0.0, 5.0, 44.0, -4.0, 1.57, 0.0], [0.0, 5.0, 46.0, -4.0, 1.57, 0.0], [0.0, 5.0, 48.0, -4.0, 1.57, 0.0], [0.0, 5.0, 50.0, -4.0, 1.57, 0.0], [0.0, 5.0, 44.0, -3.0, 1.57, 0.0], [0.0, 5.0, 46.0, -3.0, 1.57, 0.0], [0.0, 5.0, 48.0, -3.0, 1.57, 0.0], [0.0, 5.0, 50.0, -3.0, 1.57, 0.0], [0.0, 5.0, 44.0, -2.0, 1.57, 0.0], [0.0, 5.0, 46.0, -2.0, 1.57, 0.0], [0.0, 5.0, 48.0, -2.0, 1.57, 0.0], [0.0, 5.0, 50.0, -2.0, 1.57, 0.0], [0.0, 5.0, 44.0, -1.0, 1.57, 0.0], [0.0, 5.0, 46.0, -1.0, 1.57, 0.0], [0.0, 5.0, 44.0, -4.0, 1.57, 0.5], [0.0, 5.0, 46.0, -4.0, 1.57, 0.5], [0.0, 5.0, 48.0, -4.0, 1.57, 0.5], [0.0, 5.0, 50.0, -4.0, 1.57, 0.5], [0.0, 5.0, 44.0, -3.0, 1.57, 0.5], [0.0, 5.0, 46.0, -3.0, 1.57, 0.5], [0.0, 5.0, 48.0, -3.0, 1.57, 0.5], [0.0, 5.0, 50.0, -3.0, 1.57, 0.5], [0.0, 5.0, 44.0, -2.0, 1.57, 0.5], [0.0, 5.0, 46.0, -2.0, 1.57, 0.5], [0.0, 5.0, 48.0, -2.0, 1.57, 0.5], [0.0, 5.0, 50.0, -2.0, 1.57, 0.5], [0.0, 5.0, 44.0, -1.0, 1.57, 0.5], [0.0, 5.0, 46.0, -1.0, 1.57

SingleOCFState[[0.0, 5.0, 42.0, -4.0, 1.57, 0.0], [0.0, 5.0, 44.0, -4.0, 1.57, 0.0], [0.0, 5.0, 46.0, -4.0, 1.57, 0.0], [0.0, 5.0, 48.0, -4.0, 1.57, 0.0], [0.0, 5.0, 42.0, -3.0, 1.57, 0.0], [0.0, 5.0, 44.0, -3.0, 1.57, 0.0], [0.0, 5.0, 46.0, -3.0, 1.57, 0.0], [0.0, 5.0, 48.0, -3.0, 1.57, 0.0], [0.0, 5.0, 42.0, -2.0, 1.57, 0.0], [0.0, 5.0, 44.0, -2.0, 1.57, 0.0], [0.0, 5.0, 46.0, -2.0, 1.57, 0.0], [0.0, 5.0, 48.0, -2.0, 1.57, 0.0], [0.0, 5.0, 42.0, -1.0, 1.57, 0.0], [0.0, 5.0, 44.0, -1.0, 1.57, 0.0], [0.0, 5.0, 46.0, -1.0, 1.57, 0.0], [0.0, 5.0, 48.0, -1.0, 1.57, 0.0], [0.0, 5.0, 42.0, 0.0, 1.57, 0.0], [0.0, 5.0, 44.0, 0.0, 1.57, 0.0], [0.0, 5.0, 46.0, 0.0, 1.57, 0.0], [0.0, 5.0, 42.0, -4.0, 1.57, 0.5], [0.0, 5.0, 44.0, -4.0, 1.57, 0.5], [0.0, 5.0, 46.0, -4.0, 1.57, 0.5], [0.0, 5.0, 48.0, -4.0, 1.57, 0.5], [0.0, 5.0, 42.0, -3.0, 1.57, 0.5], [0.0, 5.0, 44.0, -3.0, 1.57, 0.5], [0.0, 5.0, 46.0, -3.0, 1.57, 0.5], [0.0, 5.0, 48.0, -3.0, 1.57, 0.5], [0.0, 5.0, 42.0, -2.0, 1.57, 0.5], [0.0, 5.

...

dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 8.0, 50.0, 5.0, 0.0, 0.0]], [1.0]))
action combined: LatLonAccel( 0.000,  1.000)
--------------------------POMDP high level planner----------------------- t: 11.199999999999992
EGO: x/y:49.9986666666667 / 0.0 v: 7.3100000000000005
Vehicle(2, VehicleState(VecSE2({50.000, 7.800}, 1.571), Frenet(RoadIndex({1, 0.890000}, {2, 1}), 17.800, 150.000, 0.000), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 5.000}, 0.780), Frenet(RoadIndex({1, 0.750004}, {2, 1}), 15.000, 146.000, -0.791), -0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
---------> modified: [0.0, 8.0, 50.0, 5.0, 0.0, 0.0]
dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 8.0, 50.0, 5.0, 0.0, 0.0]], [1.0]))
action combined: LatLonAccel( 0.000,  1.000)
--------------------------POMDP high level planner----------------------- t: 11.399999999999991
EGO: x/

Vehicle(3, VehicleState(VecSE2({54.000, 5.000}, 0.780), Frenet(RoadIndex({1, 0.750009}, {2, 1}), 15.000, 146.000, -0.791), -0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
---------> modified: [0.0, 10.0, 50.0, 5.0, 0.0, 0.0]
dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 10.0, 50.0, 5.0, 0.0, 0.0]], [1.0]))
action combined: LatLonAccel( 0.000,  1.000)
--------------------------POMDP high level planner----------------------- t: 13.999999999999982
EGO: x/y:74.3866666666667 / 0.0 v: 10.109999999999994
Vehicle(2, VehicleState(VecSE2({50.000, 10.000}, 1.571), Frenet(RoadIndex({1, 1.000000}, {2, 1}), 20.000, 150.000, 0.000), 0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 4.930}, 0.780), Frenet(RoadIndex({1, 0.746494}, {2, 1}), 14.930, 146.000, -0.791), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
---------> modified: [0.0, 11.0, 50.0, 5.0, 0.0, 0.0]
dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,

dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 13.0, 50.0, 5.0, 0.0, 0.0]], [1.0]))
action combined: LatLonAccel( 0.000,  1.000)
--------------------------POMDP high level planner----------------------- t: 16.799999999999972
EGO: x/y:106.61466666666662 / 0.0 v: 12.909999999999984
Vehicle(2, VehicleState(VecSE2({50.000, 10.000}, 1.571), Frenet(RoadIndex({1, 1.000000}, {2, 1}), 20.000, 150.000, 0.000), 1.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 4.649}, 0.780), Frenet(RoadIndex({1, 0.732426}, {2, 1}), 14.649, 146.000, -0.791), -0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
---------> modified: [0.0, 13.0, 50.0, 5.0, 0.0, 0.0]
dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 13.0, 50.0, 5.0, 0.0, 0.0]], [1.0]))
action combined: LatLonAccel( 0.000,  1.000)
--------------------------POMDP high level planner----------------------- t: 16.99999999999997
EG

dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 14.0, 50.0, 5.0, 0.0, 0.0]], [1.0]))
action combined: LatLonAccel( 0.000,  1.000)
--------------------------POMDP high level planner----------------------- t: 19.599999999999962
EGO: x/y:146.68266666666662 / 0.0 v: 15.709999999999974
Vehicle(2, VehicleState(VecSE2({50.000, 10.000}, 1.571), Frenet(RoadIndex({1, 1.000000}, {2, 1}), 20.000, 150.000, 0.000), 1.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))
Vehicle(3, VehicleState(VecSE2({54.000, 4.930}, 0.780), Frenet(RoadIndex({1, 0.746496}, {2, 1}), 14.930, 146.000, -0.791), -0.500), VehicleDef(PEDESTRIAN, 1.000, 1.000))
---------> modified: [0.0, 14.0, 50.0, 5.0, 0.0, 0.0]
dict-modified: Dict(-1=>SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(SingleOCFState[[0.0, 14.0, 50.0, 5.0, 0.0, 0.0]], [1.0]))
action combined: LatLonAccel( 0.000,  1.000)
--------------------------POMDP high level planner----------------------- t: 19.79999999999996
EG

10.143927402

In [5]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, CarFollowCamera(1,10.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("pomdp.mp4", film) # Write to a webm video

Frames{MIME{Symbol("image/png")}}("/tmp/tmpnPLZpZ", 0x000000000000006f, 5.0, nothing)

In [6]:


b = SingleOCFBelief([SingleOCFState(0.0, 20.0, 50.0, 5.0, 0.0, 0.0)], [1.0])  

act = action(policy, b) # policy

2-element SingleOCFAction:
 1.0
 0.0

In [7]:
ego_vehicle[end]

Vehicle(1, VehicleState(VecSE2({183.665, 0.000}, 0.000), Frenet(RoadIndex({1, 0.459162}, {1, 1}), 183.665, 0.000, 0.000), 17.910), VehicleDef(CAR, 4.000, 1.800))